In [1]:
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
import pandas as pd
import datetime
from matplotlib import pyplot as plt

import tqdm.notebook as nb
import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.ticker as mtick

import warnings
warnings.filterwarnings('ignore')

In [ ]:
plt.rc('font', family='serif', serif='Times')
# plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=8)
plt.rc('ytick', labelsize=8)
plt.rc('axes', labelsize=10)

# width as measured in inkscape
width = 3.125
height = width / 1.618

# Probability of survival from nomination for deletion

In [ ]:
all_biographies2 = pd.read_csv('data/processed/all_biographies2.csv', index_col=False)

In [ ]:
survival_data_male = all_biographies2[all_biographies2['gender']=='male']
survival_data_female = all_biographies2[all_biographies2['gender']=='female']

kmf_male = KaplanMeierFitter(label='Men')
kmf_female = KaplanMeierFitter(label='Women')


kmf_male = kmf_male.fit(durations=survival_data_male['days_before_nomination'], event_observed=survival_data_male['nominated'])
kmf_female = kmf_female.fit(durations=survival_data_female['days_before_nomination'], event_observed=survival_data_female['nominated'])

In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.18, bottom=.2, right=.99, top=.97)

kmf_male.plot(color='blue')
kmf_female.plot(color='red')
ax.set_xlabel('Days before Nomination')

ax.set_ylabel('Probability of Survival')
fig.set_size_inches(width, height)
fig.savefig('reports/figures/Male_vs_Female_Survival.png', dpi=200)
fig.show()

# Merge with Living People set 


In [ ]:
print("Loading list of Living People...")
BLP=pd.read_csv('data/petscan/Living_people.csv', index_col=False)
BLP=BLP[['title']]

print("Merging All biographies with Living people...")
all_biographies2_BLP=pd.merge(all_biographies2,BLP, left_on='page_title',right_on='title', how='left')
all_biographies2_BLP['BLP']= all_biographies2_BLP['title'].fillna(0)
all_biographies2_BLP['Entry2']=all_biographies2_BLP['Entry'].apply(lambda x: x.replace(" ","_"))
all_biographies2_BLP.loc[all_biographies2_BLP[(all_biographies2_BLP['Entry2'].isin(BLP['title'])) & (all_biographies2_BLP['BLP']==0)].index,'BLP'] =1
all_biographies2_BLP.loc[all_biographies2_BLP['BLP']!=0,'BLP']=1


print("Recoding Gender covariate...")
all_biographies2_BLP.loc[all_biographies2_BLP['gender']=='female','Female']=1
all_biographies2_BLP.loc[all_biographies2_BLP['gender']!='female','Female']=0


print("Formating date of birth and death...")
all_biographies2_BLP['birth']=all_biographies2_BLP['date_of_birth'].apply(lambda x: -1*int(x.split('-')[1]) if x[0]=='-' else( int(x.split('-')[0] ) if len(x.split('-')[0])==4 else ( int(x.split('-')[0]) if x[0]=='+' else 'no data') ) )
all_biographies2_BLP['death']=all_biographies2_BLP['date_of_death'].apply(lambda x: -1*int(x.split('-')[1]) if x[0]=='-' else( int(x.split('-')[0] ) if len(x.split('-')[0])==4 else ( int(x.split('-')[0]) if x[0]=='+' else 'no data') ) )


print("Splitting dataset based on the availability of dates of birth and death")
part_1=all_biographies2_BLP[all_biographies2_BLP['death']!='no data']
part_2=all_biographies2_BLP[(all_biographies2_BLP['birth']!='no data') & (all_biographies2_BLP['death']=='no data')]
part_3=all_biographies2_BLP[(all_biographies2_BLP['birth']=='no data') & (all_biographies2_BLP['death']=='no data')]

In [ ]:
len(all_biographies2_BLP), len(part_1),len(part_2), len(part_3), len(part_1)+len(part_2)+ len(part_3)

#### Fix the data with death data

In [ ]:
print("Part 1: Set up and process the data, focusing on entries that include a recorded date of death...")
part_1['Alive']=0

print("Splitting data based on availbility of date of birth...")
part_1A=part_1[part_1['birth']!='no data']
part_1B=part_1[part_1['birth']=='no data']

print("Removing data with wrong date of birth..")
part_1A['wrong']=part_1A.apply(lambda row: 1 if int(str(row['creation_date_original2'])[:4])< row['birth'] else 0,axis=1)
part_1A=part_1A[part_1A['wrong']==0]
part_1A=part_1A.drop(columns=['wrong'])

print("Identifying date of birth of the people who are dead...")
part_1B['birth']=part_1B['death'].apply(lambda x: x-70)

part_1=pd.concat([part_1A,part_1B])

print("Identifying the living people at the time of nomination who are deceased after 2000...")
part_1_after2000=part_1[part_1['death']>=2000]
part_1_before2000=part_1[part_1['death']<2000]
part_1_after2000['Alive']=part_1_after2000.apply(lambda row: 1 if pd.to_datetime(row['nomination_dates'],utc=True)< pd.to_datetime(str(row['death']).replace("+",""), utc=True) else 0,axis=1)



part_1=pd.concat([part_1_before2000,part_1_after2000])
print("Done with Part 1!")


In [ ]:
len(part_1)

#### Fix the data with birth date but no death date

In [ ]:
len(part_2)

In [ ]:
print("Part 2: Set up and correct the data, ensuring entries with a birth date but no death date are properly handled...\n")
print("Removing data with wrong date of birth...\n")
part_2['wrong']=part_2.apply(lambda row: 1 if int(str(row['creation_date_original2'])[:4])< row['birth'] else 0,axis=1)
part_2=part_2[part_2['wrong']==0]
part_2=part_2.drop(columns=['wrong'])

print("Identifying people born before 1907...\n")
part_2C=part_2[part_2['birth']<1907]
part_2C['Alive']=0

print("Identifying people born after 1907 and alive...\n")
part_2_notC = part_2[part_2['birth']>=1907]
part_2A = part_2_notC[part_2_notC['BLP']==1]
part_2A['Alive']=1

print("Identifying people born after 1907 and deceased...\n")
part_2B = part_2_notC[part_2_notC['BLP']==0]
dead=pd.read_csv('data/petscan/Dead_people_all.csv',error_bad_lines=False, index_col=False)
part_2B_a=part_2B[part_2B['page_title'].isin(dead['title'])].drop_duplicates(subset='page_title', keep='first')
part_2B_a['Alive']=0
part_2B_b=part_2B[~part_2B['page_title'].isin(dead['title'])].drop_duplicates(subset='page_title', keep='first')
part_2B_b['Alive']=1

part_2=pd.concat([part_2A,part_2B_a,part_2B_b,part_2C])
print("Done with Part 2!\n")

In [ ]:
len(part_2)

#### Fix the data with no birth date and no death date

In [ ]:
len(part_3)

In [ ]:
print("Identifying living people...\n")
part_3A = part_3[part_3['BLP']==1]
part_3A['Alive']=1
part_3A['is_Historical']=0
part_3B = part_3[part_3['BLP']==0]
# part_3B['Alive']=0

if cutoff 1800

In [ ]:
birth_from_begin_till_17th_century=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_from_begin_till_17th_century.csv',error_bad_lines=False, index_col=False)
birth_18thcentury=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_18thcentury.csv',error_bad_lines=False, index_col=False)


In [ ]:
historical_people = pd.concat([birth_from_begin_till_17th_century,birth_18thcentury])

In [ ]:
conteporary_people=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_19th_century_to_3rd_Millenium.csv',error_bad_lines=False, index_col=False)


If cutoff 1945

In [ ]:
birth_from_begin_till_1st_millenium=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_from_begin_till_1st_millenium.csv',error_bad_lines=False, index_col=False)
birth_2nd_millenium_without_20th_century=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_2nd_millenium_without_20th_century.csv',error_bad_lines=False, index_col=False)
birth_from1900s_to_1945=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_from1900s_to_1945.csv',error_bad_lines=False, index_col=False)



In [ ]:
historical_people = pd.concat([birth_from_begin_till_1st_millenium,birth_2nd_millenium_without_20th_century,birth_from1900s_to_1945])

In [ ]:
conteporary_people=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/birth_19th_century_to_3rd_Millenium.csv',error_bad_lines=False, index_col=False)

If cutoff 1907

In [ ]:
print("Identifying living and historical status based on birth by decades...\n")

print("collect data with date of birth before 1907...\n")
birth_from_begin_till_1st_millenium=pd.read_csv('data/petscan/birth_from_begin_till_1st_millenium.csv',error_bad_lines=False, index_col=False)
birth_2nd_millenium_without_20th_century=pd.read_csv('data/petscan/birth_2nd_millenium_without_20th_century.csv',error_bad_lines=False, index_col=False)
birth_1900s_without_07_08_09=pd.read_csv('data/petscan/birth_1900s_without_07,08,09.csv',error_bad_lines=False, index_col=False)



len(birth_from_begin_till_1st_millenium), len(birth_2nd_millenium_without_20th_century)

print("collect data with date of birth after 1907...\n")
birth_1907_08_09=pd.read_csv('data/petscan/birth_1907,08,09.csv',error_bad_lines=False, index_col=False)
birth_20th_century_without_1900s=pd.read_csv('data/petscan/birth_20th_century_without_1900s.csv',error_bad_lines=False, index_col=False)
birth_21st_century=pd.read_csv('data/petscan/birth_21st-century.csv',error_bad_lines=False, index_col=False)


historical_people = pd.concat([birth_from_begin_till_1st_millenium,birth_2nd_millenium_without_20th_century,birth_1900s_without_07_08_09])

contemporary_people = pd.concat([birth_1907_08_09,birth_20th_century_without_1900s,birth_21st_century])

print("Identifying historical people...\n")
part_3B_a=part_3B[part_3B['page_title'].isin(historical_people['title'])]
part_3B_a['Alive']=0
part_3B_a['is_Historical']=1

print("Identifying contemporary deceased people...\n")
part_3B_b=part_3B[~part_3B['page_title'].isin(historical_people['title'])]
part_3B_b_contemporary=part_3B_b[part_3B_b['page_title'].isin(contemporary_people['title'])]
dead=pd.read_csv('data/petscan/Dead_people_all.csv',error_bad_lines=False, index_col=False)
part_3B_b_contemporary_dead=part_3B_b_contemporary[part_3B_b_contemporary['page_title'].isin(dead['title'])]
part_3B_b_contemporary_dead['Alive']=0
part_3B_b_contemporary_dead['is_Historical']=0

print("Identifying data without date of birth and death information...\n")
Totally_unlnown_group = pd.concat([part_3B_b[~part_3B_b['page_title'].isin(contemporary_people['title'])],part_3B_b_contemporary[~part_3B_b_contemporary['page_title'].isin(dead['title'])]])

print("Identifying people deceased before 1907...\n")
dead_historical=pd.read_csv('data/petscan/Dead_people_historical.csv',error_bad_lines=False, index_col=False)
part_3B_unknown_historical_dead=Totally_unlnown_group[Totally_unlnown_group['page_title'].isin(dead_historical['title'])]
part_3B_unknown_historical_dead['Alive']=0
part_3B_unknown_historical_dead['is_Historical']=1


part_3_better=pd.concat([part_3A,part_3B_a,part_3B_b_contemporary_dead, part_3B_unknown_historical_dead])
not_part_3=part_3[~part_3['page_title'].isin(part_3_better['page_title'])]

print("Identifying people deceased after 1907...\n")
Dead_people_from1900_to1977=pd.read_csv('data/petscan/Dead_people_from1900_to1977.csv',error_bad_lines=False, index_col=False)
dead_contemporary_2=dead[~dead['title'].isin(pd.concat([dead_historical,Dead_people_from1900_to1977])['title'])]

print("Consider the rest of people as alive...\n")
probably_alive=not_part_3[~not_part_3['page_title'].isin(dead_contemporary_2['title'])]
probably_alive['Alive']=1
probably_alive['is_Historical']=0

part_3_better=pd.concat([part_3_better,probably_alive])



In [ ]:
Totally_unlnown_group

if cutoff 1800

In [ ]:
death_from_begin_till_18th_century=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/death_from_begin_till_18th_century.csv',error_bad_lines=False, index_col=False)
death_from_1800s_to_1870s=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/death_from_1800s_to_1870s.csv',error_bad_lines=False, index_col=False)


In [ ]:
dead_historical = pd.concat([death_from_begin_till_18th_century,death_from_1800s_to_1870s])

In [ ]:
len(dead_historical)

if cutoff 1907

In [ ]:
print("Identifying people deceased before 1907...\n")
dead_historical=pd.read_csv('data/petscan/Dead_people_historical.csv',error_bad_lines=False, index_col=False)
part_3B_unknown_historical_dead=Totally_unlnown_group[Totally_unlnown_group['page_title'].isin(dead_historical['title'])]
part_3B_unknown_historical_dead['Alive']=0
part_3B_unknown_historical_dead['is_Historical']=1


part_3_better=pd.concat([part_3A,part_3B_a,part_3B_b_contemporary_dead, part_3B_unknown_historical_dead])
not_part_3=part_3[~part_3['page_title'].isin(part_3_better['page_title'])]

In [ ]:
not_part_3['nominated'].value_counts()

if cutoff 1800

In [ ]:
death_1880s_to_3rd_millenium=pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/death_1880s_to_3rd_millenium.csv',error_bad_lines=False, index_col=False)


In [ ]:
contemporary_dead=not_part_3[not_part_3['page_title'].isin(death_1880s_to_3rd_millenium['title'])]

In [ ]:
contemporary_dead['Alive']=0
contemporary_dead['Historical']=0

In [ ]:
part_3_better

In [ ]:
probably_alive=not_part_3[~not_part_3['page_title'].isin(contemporary_dead['title'])]

In [ ]:
probably_alive

In [ ]:
len(all_biographies2_with_data)+len(probably_alive)

if cutoff 1907

In [ ]:
len(not_part_3)/len(all_biographies2_BLP)

In [ ]:
print("Identifying people deceased after 1907...\n")
Dead_people_from1900_to1977=pd.read_csv('data/petscan/Dead_people_from1900_to1977.csv',error_bad_lines=False, index_col=False)
dead_contemporary_2=dead[~dead['title'].isin(pd.concat([dead_historical,Dead_people_from1900_to1977])['title'])]

print("Consider the rest of people as alive...\n")
probably_alive=not_part_3[~not_part_3['page_title'].isin(dead_contemporary_2['title'])]
probably_alive['Alive']=1
probably_alive['is_Historical']=0

part_3_better=pd.concat([part_3_better,probably_alive])

In [ ]:
part_3_better['is_Historical'].value_counts()

In [ ]:
part_3_better['Alive'].value_counts()

#### Concat all parts 

In [ ]:
print(len(part_1), len(part_2), len(part_3_better))

In [ ]:
len(part_3_better), len(part_3)

In [ ]:
part_1[['Entry','birth','death','Alive']]

In [ ]:
part_2[['Entry','birth','death','Alive']]

In [ ]:
len(all_biographies2_BLP),len(part_1) + len(part_2) 

In [ ]:
print("Join all the data...\n")
all_biographies2_with_data=pd.concat([part_1,part_2])
all_biographies2_with_data.loc[all_biographies2_with_data[all_biographies2_with_data['birth']>=1907].index,'is_Historical']=0
all_biographies2_with_data.loc[all_biographies2_with_data[all_biographies2_with_data['birth']<1907].index,'is_Historical']=1
all_biographies2_with_data=pd.concat([all_biographies2_with_data, part_3_better]).drop_duplicates(subset='page_title', keep='first')

In [ ]:
all_biographies2_with_data['is_Historical'].value_counts()

In [ ]:
all_biographies2_with_data['is_Historical'].value_counts()

In [ ]:
all_biographies2_with_data

In [ ]:
all_biographies2_with_data = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/all_biographies2_with_data_without_history_level.csv', index_col=False)

# Study From Here 

In [ ]:
all_biographies2_with_data['Female'].value_counts()/all_biographies2_with_data['Female'].value_counts().sum()

In [ ]:
print("Identifying the feature Status for all people: a) Historical, b) Contemporary Dead, c) Contemporary Alive...\n")
all_biographies2_with_data=all_biographies2_with_data.rename(columns={'Female':'Gender'})
all_biographies2_with_data['Status']=all_biographies2_with_data.apply(lambda row: "Historical" if row['is_Historical']==1 else ("Contemporary Dead" if row['Alive']==0 else "Alive"), axis=1)
all_biographies2_with_data['Wikipedia_Age']=all_biographies2_with_data['creation_date_original2'].apply(lambda x: int(str(x)[2:4]))

In [ ]:
all_biographies2_with_data[['QID','Entry','days_before_nomination','Gender','nominated','Alive','is_Historical','Status']]

In [ ]:
all_biographies2_with_data['Status'].value_counts()/all_biographies2_with_data['Status'].value_counts().sum()

In [ ]:
all_biographies2_with_data['Status'].value_counts()/all_biographies2_with_data['Status'].value_counts().sum()

In [ ]:
all_biographies2_with_data[['Gender','Status']].value_counts()/all_biographies2_with_data[['Gender','Status']].value_counts().sum()

In [ ]:
annotation_status = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/Annotation_Status - Annotated_Status.csv', index_col=False)

In [ ]:
len(annotation_status)

In [ ]:
annotation_status.head()

In [ ]:
annotation_status.columns

In [ ]:
no_annotation_status=annotation_status[annotation_status['Alive']=='dont know']

In [ ]:
annotation_status=annotation_status[annotation_status['Alive']!='dont know']

In [ ]:
len(no_annotation_status)

In [ ]:
annotation_status['Status']=annotation_status.apply(lambda row: "Historical" if row['is_Historical']==1 else ("Contemporary Dead" if row['Alive']==0 else "Alive"), axis=1)


In [ ]:
sample_merge_evaluation=pd.merge(annotation_status[['page_title','QID','Status','birth']], all_biographies2_with_data[['page_title','QID','Status']], on = 'QID')

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(sample_merge_evaluation['Status_x'],sample_merge_evaluation['Status_y']))

In [ ]:
len(annotation_status)/len(all_biographies2_with_data)

# Cox regression analysis

In [ ]:
all_biographies2_with_data = pd.read_csv('data/processed/all_biographies2_with_data.csv', index_col=False)

In [ ]:
all_biographies2_with_data.head()

In [ ]:
fused_var_model = CoxPHFitter()
fused_var_model.fit(all_biographies2_with_data[['days_before_nomination','nominated','Gender', 'Wikipedia_Age','Status']].sample(frac=1), duration_col='days_before_nomination', event_col='nominated', formula="Gender + Wikipedia_Age + Status")

fused_var_model_interaction = CoxPHFitter()
fused_var_model_interaction.fit(all_biographies2_with_data[['days_before_nomination','nominated','Gender', 'Wikipedia_Age','Status']].sample(frac=1), duration_col='days_before_nomination', event_col='nominated', formula="Gender + Wikipedia_Age + Status + Gender*Status")


In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.58, bottom=.2, right=.99, top=.97)

fused_var_model.plot(ax=ax)
ax.set_xlim((-3,1))

fig.set_size_inches(width, height)

fig.savefig('reports/figures/Cox_prop_model.png', dpi=200)


In [ ]:
fig, ax = plt.subplots()
fig.subplots_adjust(left=.58, bottom=.2, right=.99, top=.97)

fused_var_model_interaction.plot(ax=ax)
ax.set_xlim((-3,1))

fig.set_size_inches(width, height)
fig.savefig('reports/figures/Cox_prop_model_interaction.png', dpi=200)


In [ ]:
fused_var_model.print_summary()

In [ ]:
fused_var_model_interaction.print_summary()

In [ ]:
%matplotlib inline

In [ ]:

fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True,sharey=True)
fig.subplots_adjust(left=.17, bottom=.16, right=.99, top=.97)


cycler = plt.cycler(linestyle=[':', "--", "-",':', "--", "-"],
                    color = ['blue','blue','blue','red','red', 'red'],
)
ax[0].set_prop_cycle(cycler)

fused_var_model.plot_partial_effects_on_outcome(covariates=['Gender','Status'],
                                               values=[[0,'Historical'],[0,'Contemporary Dead'],[0,'Alive'],[1,'Historical'],[1,'Contemporary Dead'],[1,'Alive']],
                                               color = ['blue','blue','blue','red','red', 'red'],
                                                      
                                               ax=ax[0]
                                                       
                                             
                                              )
ax[0].set_ylabel('Probability of Survival')
ax[0].legend(labels=['Men Historical','Men Contemporary Dead','Men Alive','Women Historical','Women Contemporary Dead','Women  Alive'], 
             loc='best', bbox_to_anchor=(3.6, 1), fontsize='x-small')


ax[1].set_prop_cycle(cycler)

fused_var_model_interaction.plot_partial_effects_on_outcome(covariates=['Gender','Status'],
                                               values=[[0,'Historical'],[0,'Contemporary Dead'],[0,'Alive'],[1,'Historical'],[1,'Contemporary Dead'],[1,'Alive']],
                                               color = ['blue','blue','blue','red','red', 'red'],
                                                      
                                               ax=ax[1]
                                                       
                                             
                                              )
ax[1].set_ylabel('Probability of Survival')
ax[1].legend('',frameon=False)
fig.text(0.3,0,'Days before Nomination', ha='left')

fig.set_size_inches(width, height)
fig.savefig('reports/figures/marginal_effect.png', bbox_inches='tight',dpi=200)


In [ ]:
from scipy.stats.distributions import chi2
def likelihood_ratio(llmin, llmax):
    return(2*(llmax-llmin))

L1 = 33195.48


L2 = 33434.42


LR = likelihood_ratio(L1,L2)


p = chi2.sf(LR, 1) # L2 has 1 DoF more than L1

print('p: %.30f' % p )

# p: 0.000000121315450836607258011741

# Experiment with Decision

In [2]:
import re

In [ ]:
# data_for_compete_risk_nominated['event'].apply(lambda x:x.replace("<span style=\"color:red;\">","")
#                                                .replace("</span>","")
#                                                .replace("<font color=green>","")
#                                                .replace("</font>","")
#                                                .replace("<font color=red>","")
#                                                .replace("<s>","")
#                                                .replace("</s>","")                                          
#                                                .replace(".","")
#                                                .replace(":","")
#                                                .replace(",","")
#                                                .replace("deleted","delete")
#                                                .replace("deletion","delete")
#                                                .replace("[[wpsoftdelete|","")
#                                                .replace("delete]]","delete")
#                                                .replace("kept","keep")
#                                                .replace("redirected","redirect")
#                                                .replace("speedily ","")
#                                                .replace("speedy ","")
#                                                .replace("soft ","")
#                                                .replace("weak ","")
#                                                .replace("snow ","")
#                                                .replace("strong ","")
#                                                .replace("boldly ","")
#                                                .replace("snowball ","")
#                                                .replace("unanimous ","")
#                                                .replace("[[wpsnow]] ","")
#                                                .replace("[[wpsnow|snow]] ","")
#                                                .replace("already ","")
#                                                .replace("procedural ","")
#                                                .replace("nomination ","")
#                                                .replace("nom ","")
#                                                .replace(", withdrawn","")
#                                                .replace(" without prejudice","")
#                                                .replace(" (non-admin closure)","")
#                                                .replace(" to delete","")
#                                                .replace(" for delete","")
#                                                .replace(" (no consensus)","")
#                                                .replace(" and cleanup","")
#                                                .replace(" and salt","")
#                                                .replace(" by nominator","")
#                                                .replace(" by nom","")
#                                                .replace(", default to keep","")
#                                                .replace("; default to keep","")
#                                                .replace("; keep","")
#                                                .replace("flagged as ","")
#                                                .replace(" (withdrawn)","")
#                                                .replace("/withdrawn","")
#                                                .replace(" all","")
#                                                .replace(" both","")
#                                                .replace(" (non-admin)","")                                          
#                                                .replace("delete (a7)","delete")
#                                                .replace("delete a7","delete")
#                                                .replace("delete as copyvio","delete")
#                                                .replace(" per delete","")
#                                                .replace("delete per delete","delete")
#                                                .replace("a7","delete")
#                                                .replace("copyvio","delete")
#                                                .replace("speedied","delete")
#                                                .replace("speedy","delete")
#                                                .replace("delete delete","delete")
#                                                .replace(" delete","delete")
#                                                .replace("delete perdelete","delete")
#                                                .replace("delete -delete","delete")
#                                                .replace("delete underdelete","delete")
#                                                .replace("delete ","delete")
#                                                .replace("deletefor now","delete")
#                                                .replace("delete-delete","delete")                                          
#                                                .replace(", no consensus","")
#                                                .replace("<font color=blue>","")
#                                                .replace(" as non-notable","")
#                                                .replace("/redirect","")
#                                                .replace("no consensus default to keep","keep")
#                                               .replace("no consensus keep","keep")
#                                               .replace("no consensus defaulting to keep","keep")
#                                               .replace("keep no consensus","keep")
#                                               .replace("keep withdrawn","keep")
#                                               .replace("close","keep")
#                                               .replace("withdrawn","keep")
#                                               .replace("keepd","keep")
#                                               .replace("procedurally keep","keep")
#                                               .replace("keep keep","keep")
#                                               .replace("keep - keep","keep")
#                                               .replace("keep and expand","keep")
#                                                .replace("withdraw","keep")
#                                                .replace("keep; keep","keep")
#                                                .replace("not delete","keep")
#                                                .replace(" keep","keep")
#                                                .replace("keep for now","keep")
#                                                .replace("[[wpsnow|keep]]","keep")
#                                                .replace("keep andkeep","keep")
#                                                .replace("'keep","keep")
#                                                .replace("keep per [[wpsnow]]","keep")
#                                                .replace("nominatorkeep","keep")
#                                                .replace("keep/keep","keep")
#                                                .replace("move","keep")
#                                                .replace("keep as moot","keep")
#                                                .replace("[[wikipediakeep|keep]]","keep")
#                                                .replace("[[wpsk|keep]]","keep")
#                                                .replace("[[wpkeep]]","keep")
#                                                .replace("moot","keep")
#                                                .replace("notdelete","keep")
#                                                .replace("keep and keep","keep")
#                                                .replace("keepal","keep")
#                                                .replace("keepd","keep")
#                                                .replace("no consensus =keep","keep")
#                                                .replace("[[wpsnow|snowball]]keep","keep")                                         
#                                               .replace("delete and redirect","redirect")
#                                               .replace("deleteand redirect","redirect")                                           
#                                               .replace("merge and redirect","merge")
#                                               .replace("merged","merge")                                           
#                                               .replace("no concensus","no consensus")
#                                               .replace(" no consensus","no consensus")
#                                               .replace("no-consensus","no consensus")                                           
#                                               .replace("no opinion","comment")
#                                               .replace("delete per delete","delete")
#                                               .replace("deleteg7","delete")                                          
#                                           )

In [3]:
def word_count(str):
    counts = dict()
    words = str.split()

    for word in words:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1

    return list(counts.values())

In [4]:
def extract_feature_for_competing_risk(afds_all,all_biographies2_with_data):
    print("Extracting features for competing risk analysis...\n")
    groups=afds_all.groupby(['page_title'])

    features_del=[]
    for i,g in nb.tqdm(groups):
        print(i)
        target= g.sort_values(by='timestamp')
        num_users=num_messages=num_words=num_seconds=0

        if len(target)>0:
            num_users = len(target['user'].unique()) +1
            num_messages = len(target)+1
            ave_num_words = (sum(target['rationals'].apply(lambda x: word_count(str(x))).sum())/(num_messages-1))+1
            num_seconds=(pd.Timestamp(target.iloc[-1]['timestamp'], unit='s')-pd.Timestamp(target.iloc[0]['timestamp'], unit='s')).total_seconds()

            if num_seconds==0:
                num_seconds=(pd.Timestamp(target.iloc[-1]['timestamp'], unit='s')-pd.Timestamp(all_biographies2_with_data[all_biographies2_with_data['page_title']==i]['nomination_dates'].iloc[0])).total_seconds()

        features_del.append([i, num_seconds,num_users,num_messages,ave_num_words])



    df_features_del=pd.DataFrame(features_del,columns=['page_title','num_seconds','num_users','num_messages','ave_num_words'])
    df_features_del=df_features_del[df_features_del['num_seconds']>0]
    return df_features_del

In [5]:
def parse_and_clean_outcomes(all_biographies2_with_data,df_features_del ):
    print("Organizing all data for competing risk analysis...\n")
    data_for_compete_risk=all_biographies2_with_data[['QID','page_title','creation_dates', 'nomination_dates','days_before_nomination','nominated','Gender', 'Wikipedia_Age','Status']]
    data_for_compete_risk['tstart']=0
    data_for_compete_risk['tstop']=data_for_compete_risk['days_before_nomination']*86400
    data_for_compete_risk['event']=data_for_compete_risk['nominated'].apply(lambda x:'not nominated' if x==0 else 'nominated')
    data_for_compete_risk['id']=data_for_compete_risk.index
    data_for_compete_risk=data_for_compete_risk.drop_duplicates(subset='id', keep='first')


    print("Organizing data of nominated afd for competing risk analysis...\n")
    data_for_compete_risk_nominated=pd.merge(data_for_compete_risk,df_features_del, on='page_title',)[['QID','page_title','creation_dates', 'nomination_dates','days_before_nomination','nominated','Gender', 'Wikipedia_Age','Status','id','num_seconds','num_users','num_messages','ave_num_words']]
    data_for_compete_risk_nominated=data_for_compete_risk_nominated.drop_duplicates(subset='id', keep='first')
    data_for_compete_risk_nominated['tstart']=data_for_compete_risk_nominated['days_before_nomination']*86400
    data_for_compete_risk_nominated['tstop']=data_for_compete_risk_nominated['tstart'] + data_for_compete_risk_nominated['num_seconds']
    data_for_compete_risk_nominated=pd.merge(data_for_compete_risk_nominated,afds_all[afds_all['action']=='Outcome'][['page_title','recommend']],on='page_title',how='left').drop_duplicates(subset='id', keep='first')
    data_for_compete_risk_nominated = data_for_compete_risk_nominated.rename(columns={'recommend':'event'})
    data_for_compete_risk_nominated=data_for_compete_risk_nominated.drop(columns='num_seconds')

    print("Cleaning the text of outcome...\n")

    data_for_compete_risk_nominated['event2']=data_for_compete_risk_nominated['event'].apply(lambda x: str(x).lower().replace("not delete","keep").replace("delete and redirect","redirect").replace("kept","keep")).apply(lambda x: re.search(r"delete|deletion|keep|no consensus|redirect|merge|withdraw|close|speedied|moot|a7|move|inclusion",x).group(0) if bool(re.search(r"delete|deletion|keep|no consensus|redirect|merge|withdraw|close|speedied|moot|a7|move|inclusion",x))==True else x ).apply(lambda x: x.replace("<span style=\"color:red;\">","" ).replace("</span>","").replace("<font color=green>","").replace("</font>","").replace("<font color=red>","").replace("withdraw","keep").replace("kept","keep"))                                                                                
    data_for_compete_risk_nominated['event2']=data_for_compete_risk_nominated['event2'].apply(lambda x: x.replace("copyvio","delete")
                                                                                                         .replace("speedied","delete")
                                                                                                         .replace("speedy","delete")
                                                                                                         .replace("deletion","delete")
                                                                                                         .replace("a7","delete")
                                                                                                         .replace("flagged as ","")
                                                                                                         .replace("move","keep")
                                                                                                         .replace("close","keep")
                                                                                                         .replace("no concensus","no consensus")
                                                                                                         .replace("no consensus","keep")
                                                                                                         .replace("moot","keep")
                                                                                                         .replace("inclusion","keep")
                                                                                                         .replace("procedural closure","keep")
                                                                                                         .replace("userfy","keep")
                                                                                                         .replace("userfied","keep")

                                                                                             )


    data_for_compete_risk_nominated=data_for_compete_risk_nominated.drop(columns='event').rename(columns={'event2':'event'})

    print("Organizing most frequent event/outcome...\n")
    data_for_compete_risk['num_users']=0
    data_for_compete_risk['num_messages']=0
    data_for_compete_risk['ave_num_words']=0

    data_for_compete_risk_all= pd.concat([data_for_compete_risk_nominated,data_for_compete_risk[data_for_compete_risk_nominated.columns]]).fillna(0)
    list_cat=list(data_for_compete_risk_all['event'].value_counts()[:6].index)
    data_for_compete_risk_all['event']=data_for_compete_risk_all['event'].apply(lambda x: x if x in list_cat else "other")
    data_for_compete_risk_all['event']=data_for_compete_risk_all['event'].apply(lambda x: x.replace(" ","-"))
    data_for_compete_risk_all=data_for_compete_risk_all.sort_values(by=['id','tstart'])
    data_for_compete_risk_all=data_for_compete_risk_all[data_for_compete_risk_all['tstop']!= data_for_compete_risk_all['tstart']]
    data_for_compete_risk_all=data_for_compete_risk_all[data_for_compete_risk_all['tstop']<=8395*86400].sort_values(by='tstop')
    
    return data_for_compete_risk_all

In [6]:
print("Loading AfD conversations...\n")
articles = pd.read_csv('data/raw/From_Begin_Afd_Conversation3.csv', index_col=False)

Loading AfD conversations...



In [7]:
print("Extracting the outcome from the conversation...\n")
articles['page_title']=articles['Entry'].apply(lambda x: str(x).replace(" ","_"))

all_biographies2_with_data = pd.read_csv('data/processed/all_biographies2_with_data.csv', index_col=False)
afds=pd.merge(articles[articles['action']=='Outcome'][['page_title','recommend']], all_biographies2_with_data, on='page_title').drop_duplicates(subset='page_title')
afds['outcome']=afds['recommend'].apply(lambda y: str(re.findall('delete|keep|merge|redirect|no consensus|d</span>elete|withdrawn|deletion|close|inclusion',str(y).lower(), flags=re.IGNORECASE)).replace("\'","").replace("[","").replace("]",""))

afds_all=pd.merge(afds[['page_title','outcome']], articles[articles['timestamp']!=-1], on='page_title')
afds_all['recommend']=afds_all['recommend'].apply(lambda x: str(x).lower())
df_features_del = extract_feature_for_competing_risk(afds_all,all_biographies2_with_data)
data_for_compete_risk_all = parse_and_clean_outcomes(all_biographies2_with_data,df_features_del)




# data_for_compete_risk_all.to_csv('csv_files/data_Gender_bias/data_survival_analysis/data_for_compete_risk_all_nominated.csv', index=False)

Extracting the outcome from the conversation...

Extracting features for competing risk analysis...



  0%|          | 0/82157 [00:00<?, ?it/s]

!dea_Star_Singer
"-cides"
"Chris_Jansing"
"Chris_Kennedy"
"Chuckawalla"
"David_Corney"
"De_Pietro"
"Frank_Davis_&_Company"
"Painter"_John_Lowe
"Weird_Al"_Yankovic
"Xavier_Pick"
"a_nowak"
"helicopter_girl"
"lady_mary_montagu"
"maury_deutsch"
$hank
'Galway'_Joe_Dolan
'Tis_the_Season_(Vince_Gill_and_Olivia_Newton-John_album)
(Howard)_Aaron_Johnson
(bah)
-minu
...Kimi_e
0010x0010
00:00:1998
1-800-Love
1-800-Therapist
1000_NICKS
100_Grand_Jin
100_Kila
101.1_THE_JUICE
1066_DC
10_Easy_Pieces_for_Piano
10thave
11:11_(Austin_Mahone_song)
121_Community_Church
150_Watts
15miles
1871_in_LGBT_rights
1907_UniFeb
1919_Market_Street
1969_Darshan
1999–2009
1M1_Records
2001_Nobel_Peace_Prize
2001_Tim_Henman_tennis_season
2008_in_gay_rights
2014_Cristina_Fernández_de_Kirchner's_speech_at_UN
20Bello
20_to_1_(darts)
22MILES
2362_(show)
24AM_Studios
26_N._Halsted
2Sides
2_in_the_AM_PM
2_the_kicker
2e_Bureau
2fresh_Records
2lane
30th_United_States_Congress_-_summary
31X
3D_Prince
3MB
3Pac
3_CD_Collector's_Se

Abdul-Azeez_Olajide_Adediran
Abdul-Basit_Agouda
Abdul-Kerim_Edilov
Abdul-Lateef_Adeniran_Akanni_Ojikutujoye_I
Abdul-Razzak_Al-Adwani
Abdul-Samadu_Musafiri
Abdul_Abbas_Ayad
Abdul_Ali_Khan
Abdul_Arif_Abdullah
Abdul_Awwal_Jaunpuri
Abdul_Aziz_(Cricketer)
Abdul_Aziz_Lutfi_Akbar
Abdul_Aziz_Omar
Abdul_Baqi_Al-Omari
Abdul_Baseer_(cricketer)
Abdul_Basit_Ahmad
Abdul_Binate
Abdul_Butt
Abdul_Gaddy
Abdul_Ghafoor_Hazarvi
Abdul_Ghafoor_Khan_Durrani
Abdul_Ghani_Pathan
Abdul_Hafeez_(chemist)
Abdul_Hafeez_Mirza
Abdul_Hai_(Delhi_cricketer)
Abdul_Hai_Arifi
Abdul_Hakim_(politician)
Abdul_Hameed_Chapra
Abdul_Hamid_Khan_(politician)
Abdul_Hassan(united_states)
Abdul_Husain_Husamuddin
Abdul_Hussein_Taslimi
Abdul_Jabar_(Bagram_detainee)
Abdul_Jabbar_(Kalat_cricketer)
Abdul_Jabbar_Al_Rifai
Abdul_Jabbar_Numan
Abdul_Kadar_Khatri
Abdul_Kahn
Abdul_Karim_Saeed_Pasha
Abdul_Khaliq_(cricketer)
Abdul_Khaliq_(philosopher)
Abdul_Khaliq_Hazara_(assassin)
Abdul_Lahij_Ahmed
Abdul_Latif_Berry
Abdul_Latif_Khalid_Cheema
Abdul_L

Adam_K._Stern
Adam_Karst
Adam_Kayoom
Adam_Kellerman
Adam_Kennedy_(programmer)
Adam_Khaki
Adam_Killian
Adam_King_(Scottish_footballer_born_1995)
Adam_Klein_(Survivor_contestant)
Adam_Kluger
Adam_Kontras
Adam_Kowalik
Adam_Krug
Adam_Kuban
Adam_Kujawski
Adam_Laughlin
Adam_Lawton
Adam_Leipzig
Adam_Leitman_Bailey
Adam_Levin
Adam_Lewis
Adam_Loewy
Adam_Logan
Adam_Lord
Adam_Lovelock
Adam_Lyons
Adam_M._Snow
Adam_Mamawala
Adam_Marczyński
Adam_Mardel
Adam_Marjoram
Adam_Martin
Adam_Mattis
Adam_Maxwell
Adam_McArthur
Adam_McCluskey
Adam_McCune
Adam_Mesh
Adam_Mills
Adam_Miron
Adam_Montgomery
Adam_Morris
Adam_Mumford
Adam_Murciano
Adam_Neate
Adam_Nelson_(footballer)
Adam_Nichols
Adam_Noble
Adam_Nunes
Adam_O'Connor
Adam_O._Brown
Adam_Ostrowski
Adam_Owen
Adam_Pacitti
Adam_Palmer
Adam_Papagan
Adam_Parfrey
Adam_Parker
Adam_Paul
Adam_Pechacek
Adam_Perly
Adam_Perry_(ice_hockey)
Adam_Pete
Adam_Peterson
Adam_Phillips_(animator)
Adam_Possamai
Adam_Powers
Adam_Preston
Adam_Quinn_(footballer)
Adam_Ragusea
Adam_Ra

...

Walter_B._Saul
Walter_Bosley
Walter_Bosshard_(photojournalist)
Walter_C._Quintard
Walter_Chala
Walter_Chin
Walter_D._O'Hearn
Walter_De_Garmo
Walter_Diaz
Walter_Dingley
Walter_Donald_Douglas
Walter_Dorn
Walter_Dröscher
Walter_Dunhan_Claus
Walter_E._Massey
Walter_E._Reno
Walter_Ehrich
Walter_Ehrlich
Walter_F._González
Walter_Figueira
Walter_Gerth
Walter_Gordon_(veteran)
Walter_Grimshaw
Walter_H._Waddell
Walter_Hilgers
Walter_Hus
Walter_I._Lawson
Walter_J._Cook
Walter_J._West
Walter_Jewell
Walter_John_Raymond
Walter_Keim
Walter_Kelleher
Walter_Kilner
Walter_Kolm
Walter_Kotaba
Walter_Lamberson
Walter_Lee_Williams
Walter_Leslie_Wilmshurst
Walter_Maestri
Walter_Maser
Walter_Masterson
Walter_Mbonu
Walter_McFadden
Walter_Mendenhall
Walter_Michael_Dickson
Walter_Mickle_Smith
Walter_Millar
Walter_Minder
Walter_Miranda
Walter_Nandalike
Walter_O'Brien
Walter_Ogilvy
Walter_Oi
Walter_Paddington
Walter_Parr
Walter_Percy_Gardiner
Walter_Pitsch
Walter_Q._Jackson
Walter_Quintus
Walter_R._Nord
Walter_ReM

Will_schillinger
Willard_C._Butcher
Willard_Keith
Willard_Prentiss
Willem_Huberts
Willem_Smit
Willem_Steenkamp
Willem_van_IJperen
Willem_van_Spronsen
Willemijn_Verkaik
Willene_Johnson
Willi_Mernyi
William_"Bill"_Douglass
William_"Bill"_McKinney
William_"Billy"_Smith
William_"Boss"_Tweed.
William_"Leffen"_Hjelte
William_'Bill'_Corbett
William_'Billy'_Burton
William_(footballer,_born_1982)
William_A._Barton
William_A._Burton
William_A._Darden
William_A._Dembski
William_A._Dodge
William_A._Koch
William_A._Stein
William_A._Tiller
William_A._Tomasso
William_A._Winder
William_A._Woodall
William_A_Mobley
William_Aalto
William_Ackman
William_Adams_(Dedham)
William_Adams_(baseball)
William_Adams_(judge)
William_Adlam
William_Aimer
William_Albert_Clark
William_Albrecht
William_Alden
William_Alexander_(architect)
William_Alfred_Gaskell
William_Aligo
William_Amelio
William_Amory_Underhill
William_Anane
William_Anthony_Nugent,_13th_Earl_of_Westmeath
William_Armstrong_(American_artist)
William_Arne_

William_Tollemache,_9th_Earl_of_Dysart
William_Towne
William_Tunberg_(artist)
William_Urban
William_Utay
William_V._Chambers
William_V._Harris
William_W._"Bill"_Carmody
William_W._Creamer
William_W._Johnstone
William_W._Parmley
William_Wacker
William_Walker_(Battle_of_Britain_pilot)
William_Wallace_(actor)
William_Wallace_Lincoln
William_Walter_(musician)
William_Ward_(police_officer)
William_Wedig
William_Wesley
William_Willes
William_Willis_(captain)
William_Willoughby,_1st_Baron_Willoughby_of_Parham
William_Wilson_Underhill
William_Wood_(United_States_Army_officer)
William_Woodard
William_Woodbridge_(educator)
William_X._Wang
William_Yosses
William_Yuan
William_george_nicholson_geddes
William_ifor_jones
William_p._o'neill
William_van_Cutsem
Williams_da_Silva_Mendonça
Williamson_Hall
Willie_Aitchison
Willie_Anderson_(rugby_coach)
Willie_Baptist
Willie_Bloomquist
Willie_Cochran
Willie_E._Gary
Willie_Flattery
Willie_H._Fuller
Willie_Harcourt-Cooze
Willie_Hastie
Willie_Herenton_(basketb

Yavuz_Nutku
Yaw_Ansong_Jr.
Yaw_Ofori_Debrah
Yawin_Smallwood
Yayoi_Yanagida
Yayuan_Liu
Yazh_Sudhakar
Yda_Hillis_Addis
Ye_Ko_Oo
Ye_Qu
Ye_Xiu
Yea-ha
Yeap_Chor_Ee
Yeasin_Arafat_(footballer)
Yedha_Lee
Yee_Jee_Tso
Yegana_Akhundova
Yegor_Omelyanenko
Yeh_Shuhua
Yehia_Abugabal
Yehoshua_Sofer
Yehua_Dennis_Wei
Yehuda_Leib_Krinsky
Yehudit_Tamir
Yeison_Asencio
Yeji_(singer)
Yejong_of_Joseon
Yekaterina_Petrovna_Zamolodchikova
Yekaterina_Volkova_(actress_and_singer)
Yelena_Chernykh
Yellow_Subs
Yellow_Sunday
Yellow_Tanabe
Yelpo
Yemane_Baria
Yemane_Niguse
Yemi_Adenuga
Yemi_Amodu
Yemi_Blaq
Yemi_Idowu
Yemi_Tella
Yeni_Kuti
Yenny
Yeo_Hoonmin
Yeoh_Kay_Ee
Yeon_Woo
Yeri_(singer)
Yericho_Christiantoko
Yerzhan_Ashikbayev
YesLogic
Yeshayahu_Ben-Aharon
Yesomi_Umolu
Yesubai
Yeung_Chan
Yevado_Okadu
Yeval
Yevgeni_Kniter
Yevgeni_Kukulyak
Yevgeni_Nikolayev
Yevgeni_Nosov_(footballer)
Yevgeni_Ponasenkov
Yevgeni_Semyonov
Yevgeny_Fyodorov_(actor)
Yevgeny_Nikitin_(actor)
Yevgeny_Zhovtis
Yevhen_Buket
Yevhen_Chernenko
Yevhen

Zack_Hample
Zack_Heart
Zack_Hexum
Zack_Kahn
Zack_Knight
Zack_Martin
Zack_Mexico
Zack_Milkovich
Zack_Mills
Zack_Novak
Zack_Ruhl
Zack_Sabre_Jnr
Zack_Sekuler
Zack_Spencer
Zack_Taylor_(celebrity_blogger)
Zack_Torquato
Zack_Werner
Zacky_Vengeance
Zaeden
Zaena_Morisho
Zafar_Ansari
Zafar_Bangash
Zafar_Karachiwala
Zafar_Mahmud
Zafar_Masud_(banker)
Zafar_Polvonov
Zafar_Shah
Zafarullah_Jan
Zafer_Gözet
Zaffar_Iqbal_Manhas
Zafir_Patel
Zafira
Zafor_Firoze
Zafri_Khan
Zafuan_Azeman
Zagonek
Zahara_Jolie-Pitt
Zaheer_Mohamed
Zahf_Paroo
Zahia_Dehar
Zahid_Ahmed_(actor)
Zahid_Al-Sheikh
Zahid_Ali_(actor)
Zahid_Ali_(politician)
Zahid_Hussain_(author)
Zahid_Hussain_(journalist)
Zahid_Mahmood
Zahida_Allen
Zahida_Zarik_Khan
Zahir_Shah_(actor)
Zahoor_Elahi_(politician)
Zahra_Abdulla
Zahra_Guliyeva
Zahra_Schreiber
Zahrad
Zaid_Ali
Zaiful_Abdul_Hakim
Zaihang
Zaijian_Jaranilla
Zain_Khan
Zain_Mahmood
Zain_Masri
Zain_Shahzad
Zainab_Abbas
Zainab_Salbi
Zainal_Shah_Alam
Zainuddin_Makhdoom_II
Zainudin_Bin_Nordin
Zainul_Ab

In [8]:
print(pd.__version__)

1.2.4


In [ ]:
all_biographies2_with_data = pd.read_csv('data/processed/all_biographies2_with_data.csv', index_col=False)


In [ ]:
df_features_del

In [ ]:
all_biographies2_with_data.columns

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['page_title']=='Gonzo_Suárez']

In [ ]:
all_biographies2_with_data[all_biographies2_with_data['page_title']=='Gonzo_Suárez']['nomination_dates'].iloc[0]

In [ ]:
data_for_compete_risk_all

In [ ]:
from dateutil import parser

In [ ]:
def convert(date_time):
    format = '%Y-%m-%d %H:%M:%S'
    datetime_str = datetime.datetime.strptime(str(date_time), format)

    return datetime_str

In [ ]:
all_biographies2

In [ ]:
data_for_compete_risk_all['creation_dates2']=data_for_compete_risk_all['creation_dates'].apply(lambda x: convert(x))

In [ ]:
data_for_compete_risk_all['nomination_dates2']=data_for_compete_risk_all['nomination_dates'].apply(lambda x: convert(x))

In [ ]:
dataframe_year=pd.DataFrame()
list_of_pf = []
iterr =1
for year in nb.tqdm(range(2004,2024,1)):
#     print(year)
    create_till=data_for_compete_risk_all[data_for_compete_risk_all['creation_dates2'] <= datetime.datetime(year, 12,31)]

    nominated=create_till[create_till['nomination_dates2'] <= datetime.datetime(year, 12,31)]
    nominated['event2']=nominated['event']
    print(nominated)

    not_nominated=create_till[create_till['nomination_dates2'] > datetime.datetime(year, 12,31)]
    not_nominated['event2']="not-nominated"
    conct=pd.concat([nominated,not_nominated])
    conct['iter']= iterr
#     print(conct)
    iterr +=1
    list_of_pf.append(conct)
    

In [ ]:
list_of_pf

In [ ]:
data_for_compete_risk_all_iter=pd.concat(list_of_pf)

In [ ]:
for i,g in nb.tqdm(data_for_compete_risk_all_iter.groupby('iter')):
    print(i)
    g.to_csv('csv_files/data_Gender_bias/data_survival_analysis/data_for_compete_risk_all_iter/iter_'+str(i)+'.csv', index=False)

In [ ]:
data_for_compete_risk_all_iter[data_for_compete_risk_all_iter['iter']==1]

In [ ]:
data_for_compete_risk_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/data_for_compete_risk_all.csv', index_col=False)

# Covariance over time(Multi-State-Model)

In [ ]:
result_compete_risk_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/data_for_compete_risk_all_iter_all_results3.csv', index_col=False)

In [ ]:
result_compete_risk_all=result_compete_risk_all[result_compete_risk_all['action']!='remove']

In [ ]:
result_compete_risk_all

In [ ]:
plt.rc('font', family='serif', serif='Times')
# plt.rc('text', usetex=True)
plt.rc('xtick', labelsize=5)
plt.rc('ytick', labelsize=4)
plt.rc('axes', labelsize=8)

# width as measured in inkscape
width = 1.625
height = width 
# / 1.618

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

In [ ]:
def plot_compete(action):
    
#     ,if action != "nominated":
    plt.rc('font', family='serif', serif='Times')
    # plt.rc('text', usetex=True)
    plt.rc('xtick', labelsize=5.5)
    plt.rc('ytick', labelsize=6)
    plt.rc('axes', labelsize=6)

    # width as measured in inkscape
    width = .72
    height = 1.618 
    # / 1.618
#     else:
#         plt.rc('font', family='serif', serif='Times')
#         plt.rc('xtick', labelsize=6)
#         plt.rc('ytick', labelsize=6)
#         plt.rc('axes', labelsize=8)

#         # width as measured in inkscape
#         width = 3.125
#         height = width / 1.618

    coef_list=result_compete_risk_all[(result_compete_risk_all['action']==action)][['year','covariate','estimate']]
    se_list=result_compete_risk_all[(result_compete_risk_all['action']==action)][['year','covariate','robust.se']]
    max_limit_upper=coef_list['estimate'].max()
    print(list(result_compete_risk_all[result_compete_risk_all['estimate']==max_limit_upper]['robust.se'])[0])
    max_limit_upper = max_limit_upper + list(result_compete_risk_all[result_compete_risk_all['estimate']==max_limit_upper]['robust.se'])[0]
    min_limit_upper=coef_list['estimate'].min()
    min_limit_upper = min_limit_upper + list(result_compete_risk_all[result_compete_risk_all['estimate']==min_limit_upper]['robust.se'])[0]


    Pvalue_list=result_compete_risk_all[(result_compete_risk_all['action']==action)][['year','covariate','p.value']]
    alpha_plot=2
    Pvalue_list['p.value2']=Pvalue_list['p.value'].apply(lambda x: 10 if x<0.05 else alpha_plot)

    
    fig, ax = plt.subplots()
    fig.subplots_adjust(left=.18, bottom=.2, right=.99, top=.97)

    
    p=ax.errorbar(coef_list[coef_list['covariate']=='Historical']['year'],coef_list[coef_list['covariate']=='Historical']['estimate'], yerr=se_list[se_list['covariate']=='Historical']['robust.se'], alpha=.5, c= 'black', linestyle='dotted', linewidth=0.5)

    for ii in range(len(coef_list['year'].unique())):
        ax.scatter(coef_list[coef_list['covariate']=='Historical']['year'].iloc[ii],coef_list[coef_list['covariate']=='Historical']['estimate'].iloc[ii],label='Historical',marker='o', s=Pvalue_list[Pvalue_list['covariate']=='Historical']['p.value2'].iloc[ii],facecolors='none',edgecolors='#a6cee3') 
    
    
    p=ax.errorbar(coef_list[coef_list['covariate']=='Contemporary Dead']['year'],coef_list[coef_list['covariate']=='Contemporary Dead']['estimate'], yerr=se_list[se_list['covariate']=='Contemporary Dead']['robust.se'], alpha= 0.5, c= 'black', linestyle='dotted', linewidth=0.5)
    for ii in range(len(coef_list['year'].unique())):
        ax.scatter(coef_list[coef_list['covariate']=='Contemporary Dead']['year'].iloc[ii],coef_list[coef_list['covariate']=='Contemporary Dead']['estimate'].iloc[ii],label='Contemporary_Dead',marker='o', s=Pvalue_list[Pvalue_list['covariate']=='Contemporary Dead']['p.value2'].iloc[ii], facecolors='none', edgecolors='#b2df8a')

    p=ax.errorbar(coef_list[coef_list['covariate']=='Gender:Contemporary Dead']['year'],coef_list[coef_list['covariate']=='Gender:Contemporary Dead']['estimate'], yerr=se_list[se_list['covariate']=='Gender:Contemporary Dead']['robust.se'], alpha= 0.5, c= 'black', linestyle='dotted', linewidth=0.5)
    for ii in range(len(coef_list['year'].unique())):
        ax.scatter(coef_list[coef_list['covariate']=='Gender:Contemporary Dead']['year'].iloc[ii],coef_list[coef_list['covariate']=='Gender:Contemporary Dead']['estimate'].iloc[ii],label='Gender:Contemporary Dead',marker='o', s=Pvalue_list[Pvalue_list['covariate']=='Gender:Contemporary Dead']['p.value2'].iloc[ii], facecolors='none', edgecolors='#33a02c')    
    
    p=ax.errorbar(coef_list[coef_list['covariate']=='Gender:Historical']['year'],coef_list[coef_list['covariate']=='Gender:Historical']['estimate'], yerr=se_list[se_list['covariate']=='Gender:Historical']['robust.se'], alpha= 0.5, c= 'black', linestyle='dotted', linewidth=0.5)
    for ii in range(len(coef_list['year'].unique())):
        ax.scatter(coef_list[coef_list['covariate']=='Gender:Historical']['year'].iloc[ii],coef_list[coef_list['covariate']=='Gender:Historical']['estimate'].iloc[ii],label='Gender:Historical',marker='o', s=Pvalue_list[Pvalue_list['covariate']=='Gender:Historical']['p.value2'].iloc[ii], facecolors='none', edgecolors='#1f78b4')    
    
    
    p=ax.errorbar(coef_list[coef_list['covariate']=='Gender']['year'],coef_list[coef_list['covariate']=='Gender']['estimate'], yerr=se_list[se_list['covariate']=='Gender']['robust.se'], alpha= 0.5, c= 'black', linestyle='dotted', linewidth=0.5)
    for ii in range(len(coef_list['year'].unique())):
        ax.scatter(coef_list[coef_list['covariate']=='Gender']['year'].iloc[ii],coef_list[coef_list['covariate']=='Gender']['estimate'].iloc[ii],label='Gender',marker='o', s=Pvalue_list[Pvalue_list['covariate']=='Gender']['p.value2'].iloc[ii], facecolors='none', edgecolors='#fb9a99')    
    
    

    labels = ['Gender','Historical','Gender $\\times$ \n Historical','Cont. Dead','Gender $\\times$ \n Cont. Dead']
    colors=['#fb9a99','#a6cee3', '#1f78b4', '#b2df8a', '#33a02c' ] 



    if action == "Delete":
        
        leg=ax.legend(labels=labels,loc='best',fontsize=4.72, handletextpad=0.05)

        for i, j in enumerate(leg.legendHandles):
            j.set_color(colors[i])
            j._sizes = [5]

#     if action == "nominated":
#         leg=ax.legend(labels=labels,loc='best', bbox_to_anchor=(0.1,0.6),fontsize=3.2)

#         for i, j in enumerate(leg.legendHandles):
#             j.set_color(colors[i])
#             j._sizes = [5]
    
   
    
    if action in ['Redirect','Merge']:
        ax.annotate(r'Ends at $'+str(round(max_limit_upper,2))+'$', xy=(2005, 2.1), xytext=(2009, 1.5), fontsize=4, arrowprops=dict(facecolor='black', shrink=1, headlength=1, width=.005,headwidth=2), )
        ax.annotate(r'Ends at $'+str(round(min_limit_upper,2))+'$', xy=(2005, -2), xytext=(2009, -1.8), fontsize=4, arrowprops=dict(facecolor='black', shrink=1, headlength=1, width=.005,headwidth=2), )

        ax.set_ylim(-2.2,2.2)
        
        
    
    
    

    ax.plot(coef_list['year'].unique(),np.zeros(len(coef_list['year'].unique())), linestyle='dotted', color='black', alpha=.8, linewidth=0.9)
    ax.set_xticks(coef_list['year'].unique().astype(int))
    ax.set_xticklabels(ax.get_xticks(), rotation = 0)
    ax.xaxis.set_major_locator(MultipleLocator(5))
#     ax.xaxis.set_major_formatter('{x:.0f}')

    # For the minor ticks, use no labels; default NullFormatter.
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.set_title('$\\^\\beta$ ('+action + ')', fontsize=8)   
    ax.tick_params(direction='out', length=5, width=1)
    ax.tick_params(which='major', pad=1)
    
    fig_dict={'Nominated':'Fig6a','Delete':'Fig6b','Keep':'Fig6c','Redirect':'Fig6d','Merge':'Fig6e'}    
    fig.tight_layout()
#     fig_dict={'nominated':'Fig6a','delete':'Fig6b','keep':'Fig6c','redirect':'Fig6d','merge':'Fig6e'}
    
    fig.set_size_inches(width, height)
#     fig.show()
    fig.savefig('figure/'+fig_dict[action]+'-decision_retrospective'+action+'.png', bbox_inches='tight',dpi=200)


In [ ]:
plot_compete('Delete')

In [ ]:
plot_compete('Keep')

In [ ]:
plot_compete('Redirect')

In [ ]:
plot_compete('Merge')

In [ ]:
plot_compete('Nominated')

# Study on the direction on Merge and Redirect

In [ ]:
data_for_compete_risk_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/data_for_compete_risk_all.csv', index_col=False)

In [ ]:
data_for_compete_risk_all = data_for_compete_risk_all_another_okay2

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['event']=='other']['Gender'].value_counts()

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['event']=='other'][['page_title']].to_csv('csv_files/data_Gender_bias/data_survival_analysis/need_extract_again.csv', index=False)

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['event']!='not-nominated']['event'].value_counts()/82533

# /data_for_compete_risk_all[data_for_compete_risk_all['event']!='not-nominated'].value_counts().sum()

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['event']!='not-nominated']['event'].value_counts()[1:].sum()

In [ ]:
data_for_compete_risk_all[data_for_compete_risk_all['event']=='other'][['page_title']]

In [ ]:
data_merge_decision=pd.merge(articles[articles['action']=='Outcome'],data_for_compete_risk_all[data_for_compete_risk_all['event']=='merge'], on='page_title').drop_duplicates(subset=['page_title','recommend'], keep='first')

In [ ]:
data_redirect_decision=pd.merge(articles[articles['action']=='Outcome'],data_for_compete_risk_all[data_for_compete_risk_all['event']=='redirect'], on='page_title').drop_duplicates(subset=['page_title','recommend'], keep='first')

In [ ]:
data_redirect_decision[['page_title','recommend','rationals']]

In [ ]:
data_merge_decision[['page_title','recommend','rationals']]

In [ ]:
data_redirect_decision['redirect_to']=data_redirect_decision['rationals'].apply(lambda x: str(x).replace("\u200e__EXPECTED_UNCONNECTED_PAGE__","")
                                                                       .replace("redirected","redirect")
                                                                       .replace("Redirect","redirect")
                                                                       .replace("REDIRECT","redirect")
                                                                       .split(".")[0].split("redirect to")[-1]
                                                                       .split("redirect into")[-1])

In [ ]:
data_redirect_decision2=data_redirect_decision[['Entry','QID','Gender','Status','page_title','redirect_to','rationals']]


In [ ]:
data_redirect_decision2[['page_title','redirect_to','rationals']][:50]

In [ ]:
data_merge_decision['merge_to']=data_merge_decision['rationals'].apply(lambda x: str(x).replace("\u200e__EXPECTED_UNCONNECTED_PAGE__","")
                                                                       .replace("merged","merge")
                                                                       .replace("Merge","merge")
                                                                       .replace("merge and redirect","merge")
                                                                       .replace("merged and redirected","merge")
                                                                       .replace("merge/redirect","merge")
                                                                       .replace("merge & redirect","merge")
                                                                       .replace("merged & redirected","merge")
                                                                       .replace("merged/redirected","merge")
                                                                       .replace("merge and Redirect","merge")
                                                                       .replace("MERGE","merge")
                                                                       .split(".")[0].split("merge to")[-1]
                                                                       .split("merge with")[-1]
                                                                       .split("merge into")[-1])

In [ ]:
data_merge_decision2=data_merge_decision[['Entry','QID','Gender','Status','page_title','merge_to','rationals']]
# [130:150]

In [ ]:
wikidata_page_id_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/wikidata_page_id_all2_merged.csv', index_col=False)


In [ ]:
wikidata_page_id_all.head()

In [ ]:
wikidata_page_id_all['Entry']=wikidata_page_id_all['page_title'].apply(lambda x: str(x).replace("_"," "))

In [ ]:
pd.merge(data_merge_decision2,wikidata_page_id_all, left_on='merge_to', right_on='Entry')

In [ ]:
data_merge_decision2

In [ ]:
import requests
import re
from lxml import etree
from bs4 import BeautifulSoup
# import mwparserfromhell
import datetime
import pandas as pd
import tqdm
from tqdm import notebook as nb
from wikidata.client import Client
import time

In [ ]:
title_list = data_merge_decision['merge_to']


In [ ]:
# title_list = data_merge_decision['merge_to']
merge_to_list = []

for i,r in nb.tqdm(data_merge_decision2[['page_title','QID','merge_to']].iterrows()):
#     if tl in list(df_gender['Entry']):
        
#         continue
#     print(tl)
    entry = r['merge_to']
    qid = 'no data'
    instances = ''
    gender = ''
    try:
        URL = 'https://en.wikipedia.org/wiki/' + entry.replace(" ","_")
        print(URL)
        page=requests.get(URL)
#         archive_num=1
        if(page.status_code==200):




        #     print(INDEX_URI2)



            soup = BeautifulSoup(page.content, "html.parser")
            qid=soup.find('li', id="t-wikibase").find('a')['href'].split("/")[-1]
#             print(qid)
            client = Client()
            entity = client.get(str(qid), load=True)

            entity

            i=entity[client.get('P31')]

            #     print(i.id)
            #     print(type(str(i.label)))
            instances=str(i.label)

            if str(i.label) == 'human':
                i2=entity[client.get('P21')]

            #             print(i.id)
            #             print(i.label)
            #             print(i2.id)
            #             print(i2.label)
                gender=str(i2.label)
#                 print(instances,gender)
    except:
        pass
    
    print([r['page_title'],r['QID'],entry,qid,instances,gender])
    merge_to_list.append([r['QID'],entry,qid,instances,gender])
#     pd.DataFrame([[entry,qid,instances,gender]]).to_csv('csv_files/data_Gender_bias/Afd From Start/Wikidata_gender_label.csv', 
#                                                         mode='a', 
#                                                         index=False,
#                                                         header=False
#                                                        )
    
    time.sleep(1)

In [ ]:
redirect_to_df = pd.DataFrame(redirect_to_list, columns=['QID','redirect_to2','QID2','instance of','Gender2']).fillna('no data')

In [ ]:
merge_to_df = pd.DataFrame(merge_to_list, columns=['QID','merge_to2','QID2','instance of','Gender2']).fillna('no data')

In [ ]:
redirect_to_df

In [ ]:
merge_to_df

In [ ]:
data_redirect_decision2_all=pd.merge(data_redirect_decision2,redirect_to_df, on='QID')

In [ ]:
data_redirect_decision2_all

In [ ]:
data_merge_decision2_all=pd.merge(data_merge_decision2,merge_to_df, on='QID')

In [ ]:
data_redirect_decision2_all.to_csv('csv_files/data_Gender_bias/data_survival_analysis/data_redirect_decision2_all.csv', index=False)

In [ ]:
data_merge_decision2_all.to_csv('csv_files/data_Gender_bias/data_survival_analysis/data_merge_decision2_all.csv', index=False)

In [ ]:
data_merge_decision2_all['instance of'].value_counts()/data_merge_decision2_all['instance of'].value_counts().sum()

# Permutation Test

In [ ]:
data_redirect_decision2_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/data_redirect_decision2_all.csv', index_col=False)

In [ ]:
data_merge_decision2_all = pd.read_csv('csv_files/data_Gender_bias/data_survival_analysis/data_merge_decision2_all.csv', index_col=False)

In [ ]:
data_redirect_decision2_all3=data_redirect_decision2_all[data_redirect_decision2_all['QID2']!='no data']

In [ ]:
data_merge_decision2_all3=data_merge_decision2_all[data_merge_decision2_all['QID2']!='no data']

In [ ]:

data_redirect_decision2_all3['Gender']= data_redirect_decision2_all3['Gender'].apply(lambda x: 'Men' if x==0 else 'Women')

data_redirect_decision2_all3['instance of2']= data_redirect_decision2_all3.apply(lambda x: x['Gender2'] if x['instance of']=='human' else 'not human', axis=1)

In [ ]:
data_merge_decision2_all3['Gender']= data_merge_decision2_all3['Gender'].apply(lambda x: 'Men' if x==0 else 'Women')
data_merge_decision2_all3['instance of2']= data_merge_decision2_all3.apply(lambda x: x['Gender2'] if x['instance of']=='human' else 'not human', axis=1)

In [ ]:
data_redirect_decision2_all3['instance of2'].value_counts()/data_redirect_decision2_all3['instance of2'].value_counts().sum()

In [ ]:
data_merge_decision2_all3['instance of2'].value_counts()/data_merge_decision2_all3['instance of2'].value_counts().sum()

## Redirect

In [ ]:
data_redirect_decision2_all4=data_redirect_decision2_all3[['page_title','Gender','redirect_to2','Gender2','instance of2']]
data_redirect_decision2_all4

In [ ]:
import warnings
warnings.filterwarnings(action='once')

In [ ]:
def permutation_test_for_outcomes(data, option):

    
    men=data[data['Gender']=='Men'][['Gender','instance of2']]
    men['Gender of Alter']=men['instance of2'].apply(lambda x: 'same' if x=='male' else('opposite' if x=='female' else 'non-bios'))
    women=data[data['Gender']=='Women'][['Gender','instance of2']]
    women['Gender of Alter']=women['instance of2'].apply(lambda x: 'same' if x=='female' else('opposite' if x=='male' else 'non-bios'))
    biography_all= pd.concat([men.drop(columns=['instance of2']),women.drop(columns=['instance of2'])])

    biography_all
    contingency_table=pd.crosstab(biography_all['Gender'], biography_all['Gender of Alter'])
    contingency_table = contingency_table.rename_axis("Group", axis="index")
    table= contingency_table.div(contingency_table.sum(axis=1), axis=0)
    print(table)

    







    n_permutations = 1000
    perm_stats = []
    alter_column = option + '_to2'
    men_expected = []
    women_expected = []
    table3 =pd.DataFrame()


    for _ in range(n_permutations):

        shuffled_data = data.copy()
        shuffled_data_part = pd.DataFrame(np.random.permutation(data[[alter_column,'instance of2']]), columns=[alter_column,'instance of2'])
        shuffled_data[alter_column]= shuffled_data_part[alter_column]
        shuffled_data['instance of2']= shuffled_data_part['instance of2']
        men=shuffled_data[shuffled_data['Gender']=='Men'][['Gender','instance of2']]
        men['Gender of Alter']=men['instance of2'].apply(lambda x: 'same' if x=='male' else('opposite' if x=='female' else 'non-bios'))
        women=shuffled_data[shuffled_data['Gender']=='Women'][['Gender','instance of2']]
        women['Gender of Alter']=women['instance of2'].apply(lambda x: 'same' if x=='female' else('opposite' if x=='male' else 'non-bios'))
        shuffled_data_biography= pd.concat([men.drop(columns=['instance of2']),women.drop(columns=['instance of2'])])
        contingency_table=pd.crosstab(shuffled_data_biography['Gender'], shuffled_data_biography['Gender of Alter'])
        contingency_table = contingency_table.rename_axis("Group", axis="index")
        table2=contingency_table.div(contingency_table.sum(axis=1), axis=0)
        table3=pd.concat([table3,table2])
    
   
    means=table3.groupby('Group').mean()
    stds=table3.groupby('Group').std()
    print(means)
    table3=table3.reset_index()
    men_tab=table3[table3['Group']=='Men']
    women_tab=table3[table3['Group']=='Women']
    
    return table,men_tab, women_tab

    
#     
    

In [ ]:
table_re,men_tab_re, women_tab_re=permutation_test_for_outcomes(data_redirect_decision2_all3, 'redirect')

In [ ]:
table_me,men_tab_me, women_tab_me=permutation_test_for_outcomes(data_merge_decision2_all3, 'merge')

In [ ]:
width = 3.125
height = width / 1.618
plt.rc('font', family='serif', serif='Times')
plt.rc('xtick', labelsize=7)
plt.rc('ytick', labelsize=6)
plt.rc('axes', labelsize=8)
# set width of bar 
barWidth = 0.6
# fig, ax = plt.subplots()
fig, ax = plt.subplots(nrows=1, ncols=2, sharex=True,sharey=True)

ax[0]=table_re.T.plot(kind='bar', ax=ax[0], color=['blue','red'], width=barWidth, alpha=.8)
xval= ax[0].get_xticks()
print(xval)
men_tab_re.boxplot( color='black', positions=[xval[0]-barWidth/4,xval[1]-barWidth/4,xval[2]-barWidth/4],grid=False, ax=ax[0])
women_tab_re.boxplot( color='black', positions=[xval[0]+barWidth/4,xval[1]+barWidth/4,xval[2]+barWidth/4],grid=False, ax=ax[0])
ticks = table_re.columns
print(ticks)
# ax[0].set_xticks(xval, ticks)
ax[0].set_title("Redirect", fontsize=10)
ax[0].legend(fontsize=8)
# plt.show()

ax[1]=table_me.T.plot(kind='bar', ax=ax[1], color=['blue','red'], width=barWidth, alpha=.8)
xval= ax[1].get_xticks()
men_tab_me.boxplot( color='black', positions=[xval[0]-barWidth/4,xval[1]-barWidth/4,xval[2]-barWidth/4],grid=False, ax=ax[1])
women_tab_me.boxplot( color='black', positions=[xval[0]+barWidth/4,xval[1]+barWidth/4,xval[2]+barWidth/4],grid=False, ax=ax[1])
ticks = table_me.columns

# ax[1].set_xticks(xval, ticks)

ax[1].set_title("Merge", fontsize=10)
ax[1].get_legend().remove()
plt.xticks(xval,ticks)
# plt.show()
ax[0].set_ylabel('Fraction')
fig.tight_layout()
fig.set_size_inches(width, height)
fig.savefig('figure/Fig5-Direction_to_merge_and_redirect.png', dpi=200, bbox_inches='tight')
fig.show()


#     xval2 = [xval[0]-width/4,xval[0]+width/4,xval[1]-width/4,xval[1]+width/4,xval[2]-width/4,xval[2]+width/4 ]
#     xmin = [xval[0]-width/2,xval[0],xval[1]-width/2,xval[1],xval[2]-width/2,xval[2] ]
#     xmax =[xval[0],xval[0]+width/2, xval[1],xval[1]+width/2,xval[2],xval[2]+width/2]

#     ymin2=[0,0,0,0,0,0]
#     ymax2=list(means[means.columns[0]])+list(means[means.columns[1]]) + list(means[means.columns[2]])

#     plt.vlines(x = xval2, ymin=ymin2, ymax=ymax2, color='black', linestyle='--')
#     plt.hlines(y=ymax2, xmin=xmin, xmax=xmax,color='black')